In [1]:
import sys
import polars as pl
from pathlib import Path

sys.path.insert(0, str(Path("../../").resolve()))
# Path relativo alla root del progetto (2 livelli sopra da strategies/example/)
data_dir = Path("../../database") 

from engine.datafeed import DataFeed, MarketData
from engine.engine import *
from engine.factor import *
from engine.plotting import *
from engine.reports import *


In [15]:
# PRICES_PATH = str(project_root / 'data')
universe = ['SX5E']
start_date='2015-01-01'
end_date='2026-12-31'

# cfg = Config(universe=universe, start_date=start_date, end_date=end_date, frequency="eod")
feed = DataFeed(prices_base=data_dir)

In [16]:
factors = [ATR(5),
            DonchianChannels(20)]
            
researcher = Researcher(factors,
                        feed,
                        start_date, end_date,
                        frequency="eod",
                        tickers=universe)

In [17]:
# fig = researcher.plot(ticker="SX5E")  # Plotta automaticamente tutti i fattori
# fig.show()

In [31]:
class TestStrategy(Strategy):

    def __init__(self) -> None:
        super().__init__()
        self.ticker = 'SX5E'
        self.factors = [
            ATR(5),
            DonchianChannels(20)
        ]
        self.position_open = False

    def on_bar(self):
        if self.period <= 1:
            return
        close = self.market_data.get(self.ticker, 0)['close'].item()
        prev_close = self.market_data.get(self.ticker, 1)['close'].item()
        dc_upper = self.market_data.get(self.ticker, 1)['dc_20_upper'].item()
        prev_dc_upper = self.market_data.get(self.ticker, 2)['dc_20_upper'].item()
        dc_lower = self.market_data.get(self.ticker, 1)['dc_20_lower'].item()
        prev_dc_lower = self.market_data.get(self.ticker, 2)['dc_20_lower'].item()
        size = self.portfolio.cash / close

        # Open position
        if close > dc_lower and prev_close < prev_dc_lower and not self.position_open:
            self.buy(self.ticker, size)
            self.position_open = True

        # Close position
        if self.position_open:
            if close < dc_upper and prev_close > prev_dc_upper:
                self.close_position(self.ticker)
                self.position_open = False
            elif close < dc_lower:
                self.close_position(self.ticker)
                self.position_open = False


In [32]:
starting_balance = 100000.

exec_model = ExecutionModel(mode="on_close", slippage_bps=0.0, commission_bps=0.0)
risk = RiskManager(max_leverage=4)
reporter = ReportWriter(out_dir="backtest_reports/notebook_demo")

In [33]:
self = TestStrategy()
self.backtest(starting_balance, start_date, end_date, "eod", universe, feed, exec_model, risk, reporter)

17:09:27 [Strategy.TestStrategy] INFO: ============================================================
17:09:27 [Strategy.TestStrategy] INFO: BACKTEST STARTED
17:09:27 [Strategy.TestStrategy] INFO: ============================================================
17:09:27 [Strategy.TestStrategy] INFO: Period: 2015-01-01 → 2026-12-31
17:09:27 [Strategy.TestStrategy] INFO: Universe: SX5E
17:09:27 [Strategy.TestStrategy] INFO: Starting Balance: $100,000.00
17:09:27 [Strategy.TestStrategy] INFO: Execution Mode: on_close
17:09:27 [Strategy.TestStrategy] INFO: Slippage: 0.0 bps | Commission: 0.0 bps
17:09:27 [Strategy.TestStrategy] INFO: ------------------------------------------------------------
17:09:27 [Strategy.TestStrategy] INFO: ORDER SUBMITTED → SX5E | LONG 28 shares | Type: MKT
17:09:27 [Strategy.TestStrategy] INFO: ORDER FILLED → SX5E | LONG 28 @ $3615.59 | Notional: $100,000.00
17:09:27 [Strategy.TestStrategy] INFO: POSITION CLOSED → SX5E
17:09:27 [Strategy.TestStrategy] INFO: ORDER SUBMI

In [34]:
analytics = StrategyAnalytics(self)

In [35]:
fig = analytics.plot_price_with_signals(
    'SX5E',
    plot_factors=['dc_20_upper', 'dc_20_lower'],
    chart_type='candlestick'  # Use candlestick chart instead of line chart
)
fig.show()


In [36]:
analytics.plot_balance("SX5E")

# DataFeed DataFrame

In [11]:
# Carica i dati di mercato usando DataFeed
df = feed.get_market_data(start_date, end_date, 'eod', universe)
print(f"Shape: {df.shape}")
print(f"\nPrime righe:")
print(df.head())


Shape: (1562, 10)

Prime righe:
shape: (5, 10)
┌────────────┬──────────┬────────┬─────────┬───┬─────────┬──────────────┬────────────────┬──────┐
│ date       ┆ time     ┆ ticker ┆ open    ┆ … ┆ close   ┆ volume       ┆ insertion_time ┆ type │
│ ---        ┆ ---      ┆ ---    ┆ ---     ┆   ┆ ---     ┆ ---          ┆ ---            ┆ ---  │
│ date       ┆ time     ┆ str    ┆ f64     ┆   ┆ f64     ┆ f64          ┆ datetime[μs]   ┆ str  │
╞════════════╪══════════╪════════╪═════════╪═══╪═════════╪══════════════╪════════════════╪══════╡
│ 2020-01-02 ┆ 01:00:00 ┆ SX5E   ┆ 3754.24 ┆ … ┆ 3793.24 ┆ 3.11856191e8 ┆ 2026-01-31     ┆ eod  │
│            ┆          ┆        ┆         ┆   ┆         ┆              ┆ 16:44:00       ┆      │
│ 2020-01-03 ┆ 01:00:00 ┆ SX5E   ┆ 3787.57 ┆ … ┆ 3773.37 ┆ 3.23515895e8 ┆ 2026-01-31     ┆ eod  │
│            ┆          ┆        ┆         ┆   ┆         ┆              ┆ 16:44:00       ┆      │
│ 2020-01-06 ┆ 01:00:00 ┆ SX5E   ┆ 3764.34 ┆ … ┆ 3752.52 ┆ 2.74219019e8

In [12]:
# feed.get_data_excel(start_date, end_date, '1m', universe)

In [13]:
# Crea MarketData dal DataFrame storico
market_data = MarketData.from_dataframe(df)
print(f"MarketData creato con {len(market_data.symbols())} simboli")
print(f"Current date: {market_data.current_date()}")
print(f"Current time: {market_data.current_time()}")


MarketData creato con 1 simboli
Current date: 2026-01-29
Current time: 01:00:00


In [14]:
# Esempi di accesso ai prezzi
print("Prezzo corrente:")
print(f"  SX5E close: ${market_data.price('SX5E', 'close'):.2f}")

print("\nPrezzo T-1 (1 bar fa):")
print(f"  SX5E close: ${market_data.price('SX5E', 'close', 1):.2f}")

print("\nUltimi 10 prezzi:")
last_10 = market_data.price('SX5E', 'close', slice(0, 10))
print(f"  Shape: {len(last_10)} valori")
print(f"  Ultimi 3: {last_10[-3:].to_list()}")


Prezzo corrente:
  SX5E close: $5891.95

Prezzo T-1 (1 bar fa):
  SX5E close: $5933.20

Ultimi 10 prezzi:
  Shape: 10 valori
  Ultimi 3: [5994.59, 5933.2, 5891.95]
